In [1]:
import warnings
warnings.filterwarnings("ignore")
%pip install nltk shap
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import logging

nltk.download('vader_lexicon')

# Inicjalizacja VADER
sia = SentimentIntensityAnalyzer()

# Funkcja do analizy sentymentu
def analyze_sentiment(sentence):
    result = sia.polarity_scores(sentence)
    return result['compound'], result['neg'], result['neu'], result['pos']

df = pd.read_csv('cleaned_trump_tweets.csv')
print(df.head())

   ---------------------------------------- 0.0/455.7 kB ? eta -:--:--
   ------------ --------------------------- 143.4/455.7 kB 2.8 MB/s eta 0:00:01
   ----------------- ---------------------- 204.8/455.7 kB 2.1 MB/s eta 0:00:01
   --------------------------- ------------ 317.4/455.7 kB 1.8 MB/s eta 0:00:01
   ----------------------------------- ---- 399.4/455.7 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------  450.6/455.7 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------  450.6/455.7 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 455.7/455.7 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   -- ------------------------------------- 0.2/2.7 MB 3.5 MB/s eta 0:00:01
   --- ------------------------------------ 0.3/2.7 MB 2.6 MB/s eta 0:00:01
   ------ --------------------------------- 0.4/2.7 MB 2.2 MB/s eta 0:00:02
   ------- -------------------------------- 0.5/2.7 MB 2.1 MB/s eta 0:00

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...


                                             content
0  sure tune watch donald trump late night david ...
1  donald trump appearing view tomorrow morning d...
2  donald trump reads top ten financial tips late...
3  new blog post celebrity apprentice finale less...
4  persona never wallflower rather build walls cl...


In [2]:
df['content'] = df['content'].astype(str)

# Analiza sentymentu dla każdego tweeta
df['compound'], df['negative'], df['neutral'], df['positive'] = zip(*df['content'].apply(analyze_sentiment))
df

,content,compound,negative,neutral,positive
0,sure tune watch donald trump late night david ...,0.4767,0.00,0.745,0.255
1,donald trump appearing view tomorrow morning d...,0.7506,0.00,0.652,0.348
2,donald trump reads top ten financial tips late...,0.5719,0.00,0.680,0.320
3,new blog post celebrity apprentice finale less...,0.0000,0.00,1.000,0.000
4,persona never wallflower rather build walls cl...,0.0000,0.00,1.000,0.000
...,...,...,...,...,...
43347,joe biden total failure government bungled eve...,-0.5106,0.32,0.680,0.000
43348,interviewed seanhannity tonight 9 00 p enjoy,0.4939,0.00,0.556,0.444
43349,pic twitter com 3lm1spbu8x,0.0000,0.00,1.000,0.000
43350,pic twitter com vpce5maduz,0.0000,0.00,1.000,0.000


In [4]:
def categorize_sentiment(compound):
    if compound >= 0.05:
        return 'positive'
    elif compound <= -0.05:
        return 'negative'
    else:
        return 'neutral'

df['sentiment'] = df['compound'].apply(categorize_sentiment)

df['number'] = [i+1 for i, _ in enumerate(df.iterrows())]
df = df[['number'] + list(df.columns[:-1])]

df.to_csv('tweets_sentiment.csv', index=False)
df



,number,number,content,compound,negative,neutral,positive
0,1,1,sure tune watch donald trump late night david ...,0.4767,0.00,0.745,0.255
1,2,2,donald trump appearing view tomorrow morning d...,0.7506,0.00,0.652,0.348
2,3,3,donald trump reads top ten financial tips late...,0.5719,0.00,0.680,0.320
3,4,4,new blog post celebrity apprentice finale less...,0.0000,0.00,1.000,0.000
4,5,5,persona never wallflower rather build walls cl...,0.0000,0.00,1.000,0.000
...,...,...,...,...,...,...,...
43347,43348,43348,joe biden total failure government bungled eve...,-0.5106,0.32,0.680,0.000
43348,43349,43349,interviewed seanhannity tonight 9 00 p enjoy,0.4939,0.00,0.556,0.444
43349,43350,43350,pic twitter com 3lm1spbu8x,0.0000,0.00,1.000,0.000
43350,43351,43351,pic twitter com vpce5maduz,0.0000,0.00,1.000,0.000
